In [1]:
import numpy as np
import pandas as pd

In [2]:
sales = pd.read_csv('../data/time_series_one.csv')

In [3]:
sales.head()

,日期,销售额
0,2017/2/17,2154
1,2017/2/18,2095
2,2017/2/19,3459
3,2017/2/20,2198
4,2017/2/21,2413


In [15]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   日期      1000 non-null   object
 1   销售额     1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [19]:
sales['日期'] = sales['日期'].astype('datetime64')

In [30]:
sales['日期'].dt.dayofweek[sales['销售额'].idxmax()]

6

In [32]:
sales['日期'].min(),sales['日期'].max()

(Timestamp('2017-02-17 00:00:00'), Timestamp('2019-11-13 00:00:00'))

In [33]:
holiday = pd.date_range(start='20170501', end='20170503').append(
          pd.date_range(start='20171001', end='20171007')).append(
          pd.date_range(start='20180215', end='20180221')).append(
          pd.date_range(start='20180501', end='20180503')).append(
          pd.date_range(start='20181001', end='20181007')).append(
          pd.date_range(start='20190204', end='20190224')).append(
          pd.date_range(start='20190501', end='20190503')).append(
          pd.date_range(start='20191001', end='20191007'))

In [39]:
sales[~sales['日期'].isin(holiday)].set_index('日期').resample('MS').sum()

,销售额
日期,
2017-02-01,31740
2017-03-01,80000
2017-04-01,74734
2017-05-01,76237
2017-06-01,80750
2017-07-01,83107
2017-08-01,85715
2017-09-01,79604
2017-10-01,61197


In [44]:
sales[sales['日期'].dt.dayofweek.isin([5,6])].set_index('日期').resample('QS').sum()

,销售额
日期,
2017-01-01,32894
2017-04-01,66692
2017-07-01,69099
2017-10-01,70384
2018-01-01,74671
2018-04-01,69950
2018-07-01,74245
2018-10-01,74699
2019-01-01,77835


In [67]:
df = sales[~sales['日期'].dt.dayofweek.isin([5,6])].set_index('日期').sort_index(ascending=False)

In [68]:
df.head()

,销售额
日期,
2019-11-13,2941
2019-11-12,2799
2019-11-11,3984
2019-11-08,3133
2019-11-07,2969


In [74]:
from datetime import datetime

In [62]:
df_temp = sales.copy()

In [63]:
sales_fri = sales.shift(4)[sales.shift(4)['日期'].dt.dayofweek==1]['销售额']
sales_mon = sales.shift(-4)[sales.shift(-4)['日期'].dt.dayofweek==5]['销售额']
df_temp.loc[sales_fri.index,['销售额']] = sales_fri
df_temp.loc[sales_mon.index,['销售额']] = sales_mon

In [71]:
df_temp

,日期,销售额
0,2017-02-17,2154.0
1,2017-02-18,2095.0
2,2017-02-19,3459.0
3,2017-02-20,2198.0
4,2017-02-21,2840.0
...,...,...
995,2019-11-09,2569.0
996,2019-11-10,2961.0
997,2019-11-11,3984.0
998,2019-11-12,2799.0


In [75]:
df_temp.loc[df_temp[df_temp['日期'].dt.year==2018]['日期'][
        df_temp[df_temp['日期'].dt.year==2018]['日期'].apply(
        lambda x:True if datetime.strptime(str(x).split()[0],'%Y-%m-%d').weekday() == 0 
        and 1 <= datetime.strptime(str(x).split()[0],'%Y-%m-%d').day <= 7 else False)].index,:]

,日期,销售额
318,2018-01-01,2863.0
353,2018-02-05,2321.0
381,2018-03-05,2705.0
409,2018-04-02,2487.0
444,2018-05-07,3204.0
472,2018-06-04,2927.0
500,2018-07-02,2574.0
535,2018-08-06,2504.0
563,2018-09-03,2483.0
591,2018-10-01,2431.0


In [81]:
sales['销售额'].rolling(window=50,min_periods=1).agg([np.mean,np.max])

,mean,amax
0,2154.000000,2154.0
1,2124.500000,2154.0
2,2569.333333,3459.0
3,2476.500000,3459.0
4,2463.800000,3459.0
...,...,...
995,2955.860000,4048.0
996,2962.060000,4048.0
997,2988.780000,4048.0
998,2989.340000,4048.0


In [82]:
def f(x):
    if len(x) == 6:
        return 1 if x[-1]>np.mean(x[:-1]) else 0
    else:
        return 0

In [90]:
sales.set_index('日期').loc[pd.date_range(start='2017-12-27',end='2018-12-31'),:].rolling(window=6,min_periods=1).agg(f)[5:]

,销售额
2018-01-01,1.0
2018-01-02,0.0
2018-01-03,0.0
2018-01-04,0.0
2018-01-05,0.0
...,...
2018-12-27,0.0
2018-12-28,1.0
2018-12-29,0.0
2018-12-30,0.0


In [91]:
def g(x):
    if len(x) == 8:
        return 1 if x[-1]>np.mean(x[:-1][pd.Series([
            False if i in [5,6] else True for i in x[:-1].index.dayofweek],index=x[:-1].index)]) else 0
    else:
        return 0

In [93]:
sales.set_index('日期').loc[pd.date_range(start='20171225',end='20181231'),:].rolling(window=8,min_periods=1).agg(g)[7:].head()

,销售额
2018-01-01,1.0
2018-01-02,0.0
2018-01-03,0.0
2018-01-04,0.0
2018-01-05,0.0
